# Import Libraries

In [ ]:
import tensorflow as tf
from urllib import request

# Download the Keras model

In [ ]:
model_url = "https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5"
keras_model_path = "bees-wasps.h5"
tflite_model_path = "bees-wasps.tflite"

# Download the Keras model
request.urlretrieve(model_url, keras_model_path)

# Convert the Keras model to TF-Lite

In [ ]:
# Load the Keras model
keras_model = tf.keras.models.load_model(keras_model_path)

# Convert Keras model to TF-Lite
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Question 1

In [1]:
# Save the TF-Lite model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# Get the size of the converted model
converted_model_size_mb = len(tflite_model) / (1024 * 1024)

# Answer to Question 1
print(f"The size of the converted model is: {converted_model_size_mb:.2f} Mb")


The size of the converted model is: 42.79 Mb


# Question 2

In [2]:
# Print the details of the TF-Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input details
input_index = interpreter.get_input_details()[0]['index']

# Get output details
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

# Answer to Question 2
print(f"The output index for this model is: {output_index}")


The output index for this model is: 13


# Preparing the Images

In [3]:
!pip install pillow

In [4]:
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Download and resize the image
image_url = "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
image = download_image(image_url)

# Assume target size based on previous homework
# Replace this with the correct target size if available
target_size = (224, 224)

# Prepare the image
resized_image = prepare_image(image, target_size)

# Print the target size
print(f"The target size for the image is: {target_size}")


The target size for the image is: (224, 224)


# Question 3

In [5]:
import numpy as np

# Convert the resized image to a numpy array
image_array = np.array(resized_image)

# Pre-process the image array
input_array = image_array / 255.0  # Normalize the pixel values to be between 0 and 1
input_array = np.expand_dims(input_array, axis=0)  # Add batch dimension

# Get the R channel value of the first pixel
r_channel_value = input_array[0, 0, 0, 0]

# Answer to Question 3
print(f"After pre-processing, the value in the first pixel, the R channel, is: {r_channel_value:.7f}")


After pre-processing, the value in the first pixel, the R channel, is: 0.9215686


# Question 4

In [8]:
# Load the TF-Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_tensor_index = interpreter.get_input_details()[0]['index']
output_tensor_index = interpreter.get_output_details()[0]['index']

# Set the input tensor
interpreter.set_tensor(input_tensor_index, input_array.astype(np.float32))

# Run the interpreter
interpreter.invoke()

# Get the output tensor
model_output = interpreter.get_tensor(output_tensor_index)

# Answer to Question 4
print(f"The output of the model is: {model_output[0][0]:.3f}")


The output of the model is: 0.659


# Question 5
![](https://i.imgur.com/rvpx5HT.png)